In [1]:
# 1. Reads the output from glc_pdb_interactingRes_between_chains.pl on 5f1b
# 2. Extracts the interfacial residues based on a distance cutoff provided by the user
# 3. Reads the MSA
# 4. For each sequence in the MSA, it reports the amino acid seq (fasta format) using only interfacial residues

In [2]:
import pandas as pd
from Bio import AlignIO
import re

In [3]:
query = 'hosthomog' #it can only be 'virus' or 'host' or 'hosthomog'
cutoff = 5

In [4]:
# Files based on user-defined input
interf_fl = '/Users/gorkalasso/Dropbox (EinsteinMed)/ebola/pdb/5f1b/X-rayAnalysis/interface_summary_4.xlsx'
interf_fl_sheet = ''
msa_fl = '' # MSA file
res_in_msa = 0 # number of first residue in MSA
ref_rec = '' # pdb accession number that will be used as reference while reading the MSA
if query == 'virus':
    print('query is virus')
    interf_fl_sheet = 'up_to_1000_gp1'
    msa_fl = '/Users/gorkalasso/Dropbox (EinsteinMed)/ebola/NPC1_GP1_binding_determinants/1_GP_seqs/3_gp1_as5f1b_1.aln'
    res_in_msa = 31
    ref_rec = '5F1B_A'
elif query == 'host':
    print('query is host')
    interf_fl_sheet = 'up_to_1000_npc1'
    msa_fl = '/Users/gorkalasso/Dropbox (EinsteinMed)/ebola/NPC1_GP1_binding_determinants/1_NPC1_seqs/4_ali_on_5f1bC_1.aln'
    res_in_msa = 373
    ref_rec = '5F1B_C'
elif query == 'hosthomog':
    print('query is hosthomog')
    interf_fl_sheet = 'up_to_1000_npc1'
    msa_fl = '/Users/gorkalasso/Dropbox (EinsteinMed)/ebola/NPC1_GP1_binding_determinants/1w1_NPC1C_homog_seqs/1_347_ali_clustalO/clustalo-I20230407-171813-0422-47029146-p2m.clustal'
    res_in_msa = 373
    ref_rec = '5F1B_C'
outfile = 'interface_' + query + '_' + str(cutoff) + '.txt'

query is hosthomog


In [5]:
# Interfacial residues
#   Getting the 3-letter to 1-letter code
df_aa = pd.read_excel('../../amino_acid_table.xlsx')
df_aa.set_index('3-letter code',inplace = True)
#  Extracting interfacial residues based on distance cutoff 
df_t = pd.read_excel(interf_fl, sheet_name = interf_fl_sheet)
df_interf = df_t[df_t['distance'] < cutoff].copy()
for index, row in df_interf.iterrows():
#     print(index)
    res = str.lower(df_interf.at[index,'res_id'])
    res = res.capitalize()
    onelet = df_aa.at[res,'1-letter code']
#     print(res, onelet)
    df_interf.at[index, '1_letter_code'] = onelet
display(df_interf)

,Chain,res_id,res_number,contact,distance,1_letter_code
30,C,TYR,420,"sidechain,backbone",3.5,Y
31,C,GLN,421,"sidechain,backbone",3.1,Q
32,C,PRO,422,"backbone,sidechain",3.7,P
33,C,TYR,423,"backbone,sidechain",3.4,Y
34,C,PRO,424,"backbone,sidechain",2.8,P
35,C,SER,425,"backbone,sidechain",3.5,S
36,C,GLY,426,backbone,3.3,G
38,C,ASP,428,"backbone,sidechain",4.3,D
111,C,ASP,501,"backbone,sidechain",2.9,D
112,C,ASP,502,"sidechain,backbone",3.4,D


In [6]:
res_numb_ls = df_interf.res_number.values.tolist()
fh = open('interfpos_' + query + '_' + str(cutoff) + '.txt', 'w')
s = ''
for item in res_numb_ls:
    item = query[0] + str(item)
    if s == '':
        s = item
    else:
        s += ', ' + item
print(s, file=fh)
fh.close()

In [7]:
# ali = bio.AlignIO.read(msa_fl, 'clustal')
ali = AlignIO.read(msa_fl, "clustal")
print(format(ali, 'clustal'))

CLUSTAL X (1.81) multiple sequence alignment


A0A8T1WIK3                          --------------------------------------------------
A0A5N5D446                          --------------------------------------------------
A0A7D5Z5E1                          --------------------------------------------------
A0A0B2VHP3                          --------------------------------------------------
A0A0V0XY86                          --------------------------------------------------
A0A0V1ARI2                          --------------------------------------------------
A0A0V0TCA5                          --------------------------------------------------
A0A0V1C9U9                          --------------------------------------------------
A0A5B7EVM2                          --------------------------------------------------
A0A8J5BY09                          ----------------------------MFFITLFIV-----------FI
A0A922I1S5                          -----------------------------------------------

In [8]:
# get reference sequence and initialize interface dictionary
ref_seq = ''
dc_interf = {}
for record in ali:
#     print(record.id, record.seq)
    if ref_rec in record.id:
        ref_seq = record.seq
        print('Extracted as reference sequence: ' + record.id)
    else:
        dc_interf[record.id] = ''
print(ref_seq)

Extracted as reference sequence: 5F1B_C
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [9]:
# get inferred interfacial residues
resnumb = res_in_msa-1
for i,c in enumerate(ref_seq):
    if c != '-':
        resnumb += 1
    else:
        continue
#     print(i, resnumb, c, sep='\t')
    if resnumb in df_interf['res_number'].values:
        idx = df_interf.index[df_interf['res_number'] == resnumb].tolist()[0]
        interfRes = df_interf.at[idx,'1_letter_code']
        print(i, resnumb, c, '->', 'Interfacial', 'index: ' + str(idx), 'resid in df: ' + interfRes, sep='\t')
        if interfRes != c:
            print('Fatal error, residues dont match')
            quit()
        for record in ali:
            if ref_rec in record.id:
                continue
#             print(record.id)
            newres = record.seq[i]
            dc_interf[record.id] = dc_interf[record.id] + newres   

1025	420	Y	->	Interfacial	index: 30	resid in df: Y
1026	421	Q	->	Interfacial	index: 31	resid in df: Q
1027	422	P	->	Interfacial	index: 32	resid in df: P
1028	423	Y	->	Interfacial	index: 33	resid in df: Y
1029	424	P	->	Interfacial	index: 34	resid in df: P
1030	425	S	->	Interfacial	index: 35	resid in df: S
1031	426	G	->	Interfacial	index: 36	resid in df: G
1039	428	D	->	Interfacial	index: 38	resid in df: D
1132	501	D	->	Interfacial	index: 111	resid in df: D
1133	502	D	->	Interfacial	index: 112	resid in df: D
1146	503	F	->	Interfacial	index: 113	resid in df: F
1147	504	F	->	Interfacial	index: 114	resid in df: F
1148	505	V	->	Interfacial	index: 115	resid in df: V
1149	506	Y	->	Interfacial	index: 116	resid in df: Y
1188	528	L	->	Interfacial	index: 138	resid in df: L


In [10]:
print(dc_interf)

{'A0A8T1WIK3': '--------YG----M', 'A0A5N5D446': '---------------', 'A0A7D5Z5E1': '---------------', 'A0A0B2VHP3': 'QSNI---QDDAGAER', 'A0A0V0XY86': 'PLDN----EEGFLIM', 'A0A0V1ARI2': 'PLDN----EEGFLLM', 'A0A0V0TCA5': 'PLDN----EEGFLLM', 'A0A0V1C9U9': 'PLDN----EEGFLLM', 'A0A5B7EVM2': '-------N-------', 'A0A8J5BY09': '-------N-------', 'A0A922I1S5': '-------D-----ND', 'A0A2S2P3P5': '--NT---S-----S-', 'A0A2H8TMY4': '--NS---T-----N-', 'A0A8X7BD69': 'KEKN---T-------', 'A0A5N5SME8': '-------D-----NN', 'W8BF01': '--NT---VDSYGYI-', 'A0A0K8UN34': '--NT---IDSNGYT-', 'A0A4Y2E852': '--N----N-----TK', 'A0A8X6QHD1': '--R----N-----IK', 'A0A8X6XH40': '--N----N-----TS', 'A0A8X6KKG1': '--N----N-----TS', 'A0A6A4WXF3': '--NT---IDSAGFEY', 'A0A8J5K2C3': '--QT---QDAFGHEL', 'A0A2P8ZEM0': '--NT---P-------', 'X2J9B0': '--NT---PEDNGFN-', 'A0A0A1WIT2': '--NT---PEDGNFE-', 'A0A034V7H3': '--NT---PEDGNFE-', 'A0A4C1YUY0': '--NT---MDEDGYE-', 'A0A146LVZ5': '--NT---VEEGNYT-', 'A0A0C9PN45': '--NT---LKEHGYV-', 'V5GT10': '--NT--

In [11]:
# Editing the headers in dictionary
dc_interf_edit = {}
for key in dc_interf.keys():
    newkey = re.sub('\/.*','',key)
    print(key, newkey)
    dc_interf_edit[newkey] = dc_interf[key]

A0A8T1WIK3 A0A8T1WIK3
A0A5N5D446 A0A5N5D446
A0A7D5Z5E1 A0A7D5Z5E1
A0A0B2VHP3 A0A0B2VHP3
A0A0V0XY86 A0A0V0XY86
A0A0V1ARI2 A0A0V1ARI2
A0A0V0TCA5 A0A0V0TCA5
A0A0V1C9U9 A0A0V1C9U9
A0A5B7EVM2 A0A5B7EVM2
A0A8J5BY09 A0A8J5BY09
A0A922I1S5 A0A922I1S5
A0A2S2P3P5 A0A2S2P3P5
A0A2H8TMY4 A0A2H8TMY4
A0A8X7BD69 A0A8X7BD69
A0A5N5SME8 A0A5N5SME8
W8BF01 W8BF01
A0A0K8UN34 A0A0K8UN34
A0A4Y2E852 A0A4Y2E852
A0A8X6QHD1 A0A8X6QHD1
A0A8X6XH40 A0A8X6XH40
A0A8X6KKG1 A0A8X6KKG1
A0A6A4WXF3 A0A6A4WXF3
A0A8J5K2C3 A0A8J5K2C3
A0A2P8ZEM0 A0A2P8ZEM0
X2J9B0 X2J9B0
A0A0A1WIT2 A0A0A1WIT2
A0A034V7H3 A0A034V7H3
A0A4C1YUY0 A0A4C1YUY0
A0A146LVZ5 A0A146LVZ5
A0A0C9PN45 A0A0C9PN45
V5GT10 V5GT10
A0A3L6GBV5 A0A3L6GBV5
A0A371G4H6 A0A371G4H6
A0A1D1YJ54 A0A1D1YJ54
A0A438ETF0 A0A438ETF0
A0A6G1S848 A0A6G1S848
A0A151MMK0 A0A151MMK0
W0Z650 W0Z650
A0A2B4RDP3 A0A2B4RDP3
T2MJG9 T2MJG9
A0A8J5AP25 A0A8J5AP25
A0A8J9Z446 A0A8J9Z446
A0A674N048 A0A674N048
A0A6F9DN43 A0A6F9DN43
V8P1N0 V8P1N0
D_russellii D_russellii
A0A6P9CQR9 A0A6P9CQR9
A0A670YBJ4 A

In [12]:
fl = open(outfile,'w')
for key, val in dc_interf_edit.items():
    print(f'>{key}', file=fl)
    print(val, file=fl)
fl.close()

In [13]:
print('Finito')

Finito
